# MGMT 467 — Prompt-Driven Lab (with Commented Examples)
## Kaggle ➜ Google Cloud Storage ➜ BigQuery ➜ Data Quality (DQ)

**How to use this notebook**
- Each section gives you a **Build Prompt** to paste into Gemini/Vertex AI (or Gemini in Colab).
- Below each prompt, you’ll see a **commented example** of what a good LLM answer might look like.
- **Do not** just uncomment and run. Use the prompt to generate your own code, then compare to the example.
- After every step, run the **Verification Prompt**, and write the **Reflection** in Markdown.

> Goal today: Download the Netflix dataset (Kaggle) → Stage on GCS → Load into BigQuery → Run DQ profiling (missingness, duplicates, outliers, anomaly flags).


### Academic integrity & LLM usage
- Use the prompts here to generate your own code cells.
- Read concept notes and write the reflection answers in your own words.
- Keep credentials out of code. Upload `kaggle.json` when asked.


## Learning objectives
1) Explain **why** we stage data in GCS and load it to BigQuery.  
2) Build an **idempotent**, auditable pipeline.  
3) Diagnose **missingness**, **duplicates**, and **outliers** and justify cleaning choices.  
4) Connect DQ decisions to **business/ML impact**.


## 0) Environment setup — What & Why
Authenticate Colab to Google Cloud so we can use `gcloud`, GCS, and BigQuery. Set **PROJECT_ID** and **REGION** once for consistency (cost/latency).

### Build Prompt (paste to LLM)
You are my cloud TA. Generate a single **Colab code cell** that:
1) Authenticates to Google Cloud in Colab,  
2) Prompts for `PROJECT_ID` via `input()` and sets `REGION="us-central1"` (editable),  
3) Exports `GOOGLE_CLOUD_PROJECT`,  
4) Runs `gcloud config set project $GOOGLE_CLOUD_PROJECT`,  
5) Prints both values. Add 2–3 comments explaining what/why.
End with a comment: `# Done: Auth + Project/Region set`.


In [1]:
# # EXAMPLE (from LLM) — Auth + Project/Region (commented; write your own cell using the prompt)
from google.colab import auth
auth.authenticate_user()

import os
PROJECT_ID = input("Enter your GCP Project ID: ").strip()
REGION = "us-central1"  # keep consistent; change if instructed
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
print("Project:", PROJECT_ID, "| Region:", REGION)

# Set active project for gcloud/BigQuery CLI
!gcloud config set project $GOOGLE_CLOUD_PROJECT
!gcloud config get-value project
# Done: Auth + Project/Region set

Enter your GCP Project ID: mgmt-467-2500
Project: mgmt-467-2500 | Region: us-central1
Updated property [core/project].
mgmt-467-2500


### Verification Prompt
Generate a short cell that prints the active project using `gcloud config get-value project` and echoes the `REGION` you set.


**Reflection:** Why do we set `PROJECT_ID` and `REGION` at the top? What can go wrong if we don’t?

## 1) Kaggle API — What & Why
Use Kaggle CLI for reproducible downloads. Store `kaggle.json` at `~/.kaggle/kaggle.json` with `0600` permissions to protect secrets.

### Build Prompt
Generate a **single Colab code cell** that:
- Prompts me to upload `kaggle.json`,
- Saves to `~/.kaggle/kaggle.json` with `0600` permissions,
- Prints `kaggle --version`.
Add comments about security and reproducibility.


In [13]:
# # EXAMPLE (from LLM) — Kaggle setup (commented)
from google.colab import files
print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
uploaded = files.upload()
# #
import os
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])
    os.chmod('/root/.kaggle/kaggle.json', 0o600)  # owner-only

!kaggle --version

Upload your kaggle.json (Kaggle > Account > Create New API Token)


Saving kaggle.json to kaggle (2).json
Kaggle API 1.7.4.5


### Verification Prompt
Generate a one-liner that runs `kaggle --help | head -n 20` to show the CLI is ready.


**Reflection:** Why require strict `0600` permissions on API tokens? What risks are we avoiding?

In [ ]:
!kaggle --help | head -n 20

usage: kaggle [-h] [-v] [-W]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...

options:
  -h, --help            show this help message and exit
  -v, --version         Print the Kaggle API version
  -W, --no-warn         Disable out-of-date API version warning

commands:
  {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        kernels {list, files, init, push, pull, output, status}
                        models {instances, get, list, init, create, delete, update}
                        models instances {versions, get, files, init, create, delete, update}
                        models instances versions {init, create, download, delete, files}
                        config {view, set,

## 2) Download & unzip dataset — What & Why
Keep raw files under `/content/data/raw` for predictable paths and auditing.
**Dataset:** `sayeeduddin/netflix-2025user-behavior-dataset-210k-records`

### Build Prompt
Generate a **Colab code cell** that:
- Creates `/content/data/raw`,
- Downloads the dataset to `/content/data` with Kaggle CLI,
- Unzips into `/content/data/raw` (overwrite OK),
- Lists all CSVs with sizes in a neat table.
Include comments describing each step.


In [14]:
# # EXAMPLE (from LLM) — Download & unzip (commented)
!mkdir -p /content/data/raw
!kaggle datasets download -d sayeeduddin/netflix-2025user-behavior-dataset-210k-records -p /content/data
!unzip -o /content/data/*.zip -d /content/data/raw
# # # List CSV inventory
!ls -lh /content/data/raw/*.csv

Dataset URL: https://www.kaggle.com/datasets/sayeeduddin/netflix-2025user-behavior-dataset-210k-records
License(s): CC0-1.0
netflix-2025user-behavior-dataset-210k-records.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/data/netflix-2025user-behavior-dataset-210k-records.zip
  inflating: /content/data/raw/README.md  
  inflating: /content/data/raw/movies.csv  
  inflating: /content/data/raw/recommendation_logs.csv  
  inflating: /content/data/raw/reviews.csv  
  inflating: /content/data/raw/search_logs.csv  
  inflating: /content/data/raw/users.csv  
  inflating: /content/data/raw/watch_history.csv  
-rw-r--r-- 1 root root 114K Aug  2 19:36 /content/data/raw/movies.csv
-rw-r--r-- 1 root root 4.5M Aug  2 19:36 /content/data/raw/recommendation_logs.csv
-rw-r--r-- 1 root root 1.8M Aug  2 19:36 /content/data/raw/reviews.csv
-rw-r--r-- 1 root root 2.2M Aug  2 19:36 /content/data/raw/search_logs.csv
-rw-r--r-- 1 root root 1.6M Aug  2 1

### Verification Prompt
Generate a snippet that asserts there are exactly **six** CSV files and prints their names.


**Reflection:** Why is keeping a clean file inventory (names, sizes) useful downstream?

To keep track of what files are currently in there and how much space each is taking up or how much space is left

## 3) Create GCS bucket & upload — What & Why
Stage in GCS → consistent, versionable source for BigQuery loads. Bucket names must be **globally unique**.

### Build Prompt
Generate a **Colab code cell** that:
- Creates a unique bucket in `${REGION}` (random suffix),
- Saves name to `BUCKET_NAME` env var,
- Uploads all CSVs to `gs://$BUCKET_NAME/netflix/`,
- Prints the bucket name and explains staging benefits.


In [15]:
# # EXAMPLE (from LLM) — GCS staging (commented)
import uuid, os
bucket_name = f"mgmt467-netflix-{uuid.uuid4().hex[:8]}"
os.environ["BUCKET_NAME"] = bucket_name
!gcloud storage buckets create gs://$BUCKET_NAME --location="us-central1"
!gcloud storage cp /content/data/raw/* gs://$BUCKET_NAME/netflix/
print("Bucket:", bucket_name)
# # # Verify contents
!gcloud storage ls gs://$BUCKET_NAME/netflix/

Creating gs://mgmt467-netflix-d8f4628a/...
Copying file:///content/data/raw/movies.csv to gs://mgmt467-netflix-d8f4628a/netflix/movies.csv
Copying file:///content/data/raw/README.md to gs://mgmt467-netflix-d8f4628a/netflix/README.md
Copying file:///content/data/raw/recommendation_logs.csv to gs://mgmt467-netflix-d8f4628a/netflix/recommendation_logs.csv
Copying file:///content/data/raw/reviews.csv to gs://mgmt467-netflix-d8f4628a/netflix/reviews.csv
Copying file:///content/data/raw/search_logs.csv to gs://mgmt467-netflix-d8f4628a/netflix/search_logs.csv
Copying file:///content/data/raw/users.csv to gs://mgmt467-netflix-d8f4628a/netflix/users.csv
Copying file:///content/data/raw/watch_history.csv to gs://mgmt467-netflix-d8f4628a/netflix/watch_history.csv

Average throughput: 46.3MiB/s
Bucket: mgmt467-netflix-d8f4628a
gs://mgmt467-netflix-d8f4628a/netflix/README.md
gs://mgmt467-netflix-d8f4628a/netflix/movies.csv
gs://mgmt467-netflix-d8f4628a/netflix/recommendation_logs.csv
gs://mgmt467-n

### Verification Prompt
Generate a snippet that lists the `netflix/` prefix and shows object sizes.


In [16]:
import uuid
import os

# Generate a new unique bucket name
bucket_name = f"mgmt467-netflix-{uuid.uuid4().hex[:8]}"
os.environ["BUCKET_NAME"] = bucket_name

# Attempt to create the bucket
!gcloud storage buckets create gs://$BUCKET_NAME --location="us-central1"

# Check if bucket creation was successful before proceeding
# Upload the files
!gcloud storage cp /content/data/raw/* gs://$BUCKET_NAME/netflix/

# List files with sizes
!gcloud storage ls gs://$BUCKET_NAME/netflix/ --human-readable

Creating gs://mgmt467-netflix-bfe91a4e/...
Copying file:///content/data/raw/movies.csv to gs://mgmt467-netflix-bfe91a4e/netflix/movies.csv
Copying file:///content/data/raw/README.md to gs://mgmt467-netflix-bfe91a4e/netflix/README.md
Copying file:///content/data/raw/recommendation_logs.csv to gs://mgmt467-netflix-bfe91a4e/netflix/recommendation_logs.csv
Copying file:///content/data/raw/reviews.csv to gs://mgmt467-netflix-bfe91a4e/netflix/reviews.csv
Copying file:///content/data/raw/search_logs.csv to gs://mgmt467-netflix-bfe91a4e/netflix/search_logs.csv
Copying file:///content/data/raw/users.csv to gs://mgmt467-netflix-bfe91a4e/netflix/users.csv
Copying file:///content/data/raw/watch_history.csv to gs://mgmt467-netflix-bfe91a4e/netflix/watch_history.csv

Average throughput: 23.6MiB/s
ERROR: (gcloud.storage.ls) unrecognized arguments: --human-readable (did you mean '--readable-sizes'?) 

To search the help text of gcloud commands, run:
  gcloud help -- SEARCH_TERMS


**Reflection:** Name two benefits of staging in
GCS vs loading directly from local Colab.

Scalability and Performance: GCS is a highly scalable and performant object storage service. Loading data into BigQuery from GCS is generally faster and more efficient, especially for large datasets, as it leverages Google Cloud's internal network and infrastructure.

Reliability and Durability: Data stored in GCS is highly durable and available. It provides built-in redundancy and protection against data loss, ensuring your data is safe and accessible for BigQuery loading and other processes.

## 4) BigQuery dataset & loads — What & Why
Create dataset `netflix` and load six CSVs with **autodetect** for speed (we’ll enforce schemas later).

### Build Prompt (two cells)
**Cell A:** Create (idempotently) dataset `netflix` in US multi-region; if it exists, print a friendly message.  
**Cell B:** Load tables from `gs://$BUCKET_NAME/netflix/`:
`users, movies, watch_history, recommendation_logs, search_logs, reviews`
with `--skip_leading_rows=1 --autodetect --source_format=CSV`.
Finish with row-count queries for each table.


In [17]:
# # EXAMPLE (from LLM) — BigQuery dataset (commented)
DATASET="netflix"
# # # Attempt to create; ignore if exists
!bq --location=US mk -d --description "MGMT467 Netflix dataset" $DATASET || echo "Dataset may already exist."

Dataset 'mgmt-467-2500:netflix' successfully created.


In [18]:
# # EXAMPLE (from LLM) — Load tables (commented)
tables = {
   "users": "users.csv",
   "movies": "movies.csv",
   "watch_history": "watch_history.csv",
   "recommendation_logs": "recommendation_logs.csv",
   "search_logs": "search_logs.csv",
   "reviews": "reviews.csv",
 }
import os
DATASET = "netflix" # Assuming DATASET is set in a previous cell
for tbl, fname in tables.items():
   src = f"gs://{os.environ['BUCKET_NAME']}/netflix/{fname}"
   print("Loading", tbl, "from", src)
   # Corrected bq load syntax: destination_table, source_uri
   !bq load --skip_leading_rows=1 --autodetect --source_format=CSV {DATASET}.{tbl} {src}

# # # Row counts
for tbl in tables.keys():
    # Corrected bq query syntax with escaped backticks
    query = f"SELECT '{tbl}' AS table_name, COUNT(*) AS n FROM `{os.environ['GOOGLE_CLOUD_PROJECT']}.netflix.{tbl}`"
    !bq query --nouse_legacy_sql "{query}"

Loading users from gs://mgmt467-netflix-bfe91a4e/netflix/users.csv
Waiting on bqjob_r40ed60d6651f5a62_0000019a0396c041_1 ... (1s) Current status: DONE   
Loading movies from gs://mgmt467-netflix-bfe91a4e/netflix/movies.csv
Waiting on bqjob_r544b0161e59c4576_0000019a0396d57b_1 ... (1s) Current status: DONE   
Loading watch_history from gs://mgmt467-netflix-bfe91a4e/netflix/watch_history.csv
Waiting on bqjob_r7a6e72bd2afb3d56_0000019a0396ec0a_1 ... (2s) Current status: DONE   
Loading recommendation_logs from gs://mgmt467-netflix-bfe91a4e/netflix/recommendation_logs.csv
Waiting on bqjob_r243fdeb65c6a6461_0000019a039704cc_1 ... (1s) Current status: DONE   
Loading search_logs from gs://mgmt467-netflix-bfe91a4e/netflix/search_logs.csv
Waiting on bqjob_r2656d82df0663b86_0000019a0397192c_1 ... (1s) Current status: DONE   
Loading reviews from gs://mgmt467-netflix-bfe91a4e/netflix/reviews.csv
Waiting on bqjob_r600d529cec30eb3a_0000019a03972ed4_1 ... (1s) Current status: DONE   
/bin/bash: lin

### Verification Prompt
Generate a single query that returns `table_name, row_count` for all six tables in `${GOOGLE_CLOUD_PROJECT}.netflix`.


In [19]:
%%bigquery
SELECT 'users' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-2500.netflix.users`
UNION ALL
SELECT 'movies' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-2500.netflix.movies`
UNION ALL
SELECT 'watch_history' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-2500.netflix.watch_history`
UNION ALL
SELECT 'recommendation_logs' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-2500.netflix.recommendation_logs`
UNION ALL
SELECT 'search_logs' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-2500.netflix.search_logs`
UNION ALL
SELECT 'reviews' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-2500.netflix.reviews`

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,row_count
0,movies,1040
1,users,10300
2,search_logs,26500
3,reviews,15450
4,recommendation_logs,52000
5,watch_history,105000


**Reflection:** When is `autodetect` acceptable? When should you enforce explicit schemas and why?

Autodetect is acceptable for quick exploration or small datasets, but for production pipelines and large datasets, always enforce explicit schemas to ensure data consistency, quality, and maintainability. Explicit schemas prevent unexpected errors from schema changes and are crucial for reliable data governance.

## 5) Data Quality (DQ) — Concepts we care about
- **Missingness** (MCAR/MAR/MNAR). Impute vs drop. Add `is_missing_*` indicators.
- **Duplicates** (exact vs near). Double-counted engagement corrupts labels & KPIs.
- **Outliers** (IQR). Winsorize/cap vs robust models. Always **flag** and explain.
- **Reproducibility**. Prefer `CREATE OR REPLACE` and deterministic keys.


### 5.1 Missingness (users) — What & Why
Measure % missing and check if missingness depends on another variable (MAR) → potential bias & instability.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Total rows and % missing in `region`, `plan_tier`, `age_band` from `users`.
2) `% plan_tier missing by region` ordered descending. Add comments on MAR.


In [ ]:
%%bigquery
-- Users: Total rows and % missing per column (region, plan_tier, age_band)
WITH base AS (
  SELECT COUNT(*) n,
         COUNTIF(state_province IS NULL) miss_region,
         COUNTIF(subscription_plan IS NULL) miss_plan,
         COUNTIF(age IS NULL) miss_age
  FROM `mgmt-467-2500.netflix.users`
)
SELECT n,
       ROUND(SAFE_DIVIDE(100*miss_region, n),2) AS pct_missing_region,
       ROUND(SAFE_DIVIDE(100*miss_plan, n),2)   AS pct_missing_plan_tier,
       ROUND(SAFE_DIVIDE(100*miss_age, n),2)    AS pct_missing_age_band
FROM base;

Query is running:   0%|          |

Downloading:   0%|          |

,n,pct_missing_region,pct_missing_plan_tier,pct_missing_age_band
0,51500,0.0,0.0,11.93


In [ ]:
%%bigquery
-- Users: % plan_tier missing by region (examining MAR)
-- If the percentage of missing values for a column varies significantly across categories
-- of another variable (like region), it might indicate Missing At Random (MAR).
SELECT state_province,
       COUNT(*) AS n,
       ROUND(SAFE_DIVIDE(100*COUNTIF(subscription_plan IS NULL), COUNT(*)),2) AS pct_missing_plan_tier
FROM `mgmt-467-2500.netflix.users`
GROUP BY state_province
ORDER BY pct_missing_plan_tier DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,state_province,n,pct_missing_plan_tier
0,New Brunswick,1440,0.0
1,Saskatchewan,1430,0.0
2,Texas,1880,0.0
3,Virginia,1785,0.0
4,Alberta,1550,0.0
5,British Columbia,1545,0.0
6,California,1815,0.0
7,Georgia,1795,0.0
8,Manitoba,1495,0.0
9,Massachusetts,1725,0.0


### Verification Prompt
Generate a query that prints the three missingness percentages from (1), rounded to two decimals.


In [ ]:
%%bigquery
WITH base AS (
  SELECT COUNT(*) n,
         COUNTIF(state_province IS NULL) miss_region,
         COUNTIF(subscription_plan IS NULL) miss_plan,
         COUNTIF(age IS NULL) miss_age
  FROM `mgmt-467-2500.netflix.users`
)
SELECT n,
       ROUND(SAFE_DIVIDE(100*miss_region, n),2) AS pct_missing_region,
       ROUND(SAFE_DIVIDE(100*miss_plan, n),2)   AS pct_missing_plan_tier,
       ROUND(SAFE_DIVIDE(100*miss_age, n),2)    AS pct_missing_age_band
FROM base;

Query is running:   0%|          |

Downloading:   0%|          |

,n,pct_missing_region,pct_missing_plan_tier,pct_missing_age_band
0,51500,0.0,0.0,11.93


**Reflection:** Which columns are most missing? Hypothesize MCAR/MAR/MNAR and why.

### 5.2 Duplicates (watch_history) — What & Why
Find exact duplicate interaction records and keep **one best** per group (deterministic policy).

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Report duplicate groups on `(user_id, movie_id, event_ts, device_type)` with counts (top 20).
2) Create table `watch_history_dedup` that keeps one row per group (prefer higher `progress_ratio`, then `minutes_watched`). Add comments.


In [ ]:
%%bigquery
-- Report duplicate groups on (user_id, movie_id, event_ts, device_type) with counts (top 20)
SELECT user_id, movie_id, device_type, COUNT(*) AS dup_count
FROM `mgmt-467-2500.netflix.watch_history`
GROUP BY user_id, movie_id, device_type
HAVING dup_count > 1
ORDER BY dup_count DESC
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,movie_id,device_type,dup_count
0,user_03310,movie_0640,Smart TV,20
1,user_00391,movie_0893,Laptop,20
2,user_01870,movie_0844,Laptop,15
3,user_02138,movie_0729,Laptop,15
4,user_00249,movie_0203,Laptop,15
5,user_07529,movie_0686,Laptop,15
6,user_08177,movie_0593,Laptop,15
7,user_06554,movie_0505,Laptop,15
8,user_02028,movie_0037,Desktop,15
9,user_06417,movie_0590,Laptop,15


In [ ]:
%%bigquery
-- Create table watch_history_dedup that keeps one row per group (prefer higher progress_ratio, then minutes_watched)
-- Use a window function to assign a rank to each row within a partition of the grouping columns.
-- The ORDER BY clause in the window function defines the "keep-one" policy.
CREATE OR REPLACE TABLE `mgmt-467-2500.netflix.watch_history_dedup` AS
SELECT * EXCEPT(rk) FROM (
  SELECT h.*,
         ROW_NUMBER() OVER (
           PARTITION BY user_id, movie_id, device_type
           ORDER BY progress_percentage DESC, watch_duration_minutes DESC
         ) AS rk
  FROM `mgmt-467-2500.netflix.watch_history` h
)
WHERE rk = 1;

Query is running:   0%|          |

""


### Verification Prompt
Generate a before/after count query comparing raw vs `watch_history_dedup`.


In [ ]:
%%bigquery
-- Compare row counts before and after deduplication
SELECT 'watch_history_raw' AS table_name, COUNT(*) AS row_count
FROM `mgmt-467-2500.netflix.watch_history`
UNION ALL
SELECT 'watch_history_dedup' AS table_name, COUNT(*) AS row_count
FROM `mgmt-467-2500.netflix.watch_history_dedup`;

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,row_count
0,watch_history_dedup,99878
1,watch_history_raw,525000


**Reflection:** Why do duplicates arise (natural vs system-generated)? How do they corrupt labels and KPIs?

### 5.3 Outliers (minutes_watched) — What & Why
Estimate extreme values via IQR; report % outliers; **winsorize** to P01/P99 for robustness while also **flagging** extremes.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Compute IQR bounds for `minutes_watched` on `watch_history_dedup` and report % outliers.
2) Create `watch_history_robust` with `minutes_watched_capped` capped at P01/P99; return quantile summaries before/after.


In [ ]:
%%bigquery
WITH dist AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(1)] AS q1,
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(3)] AS q3
  FROM `mgmt-467-2500.netflix.watch_history_dedup`
),
bounds AS (
  SELECT q1, q3, (q3-q1) AS iqr,
         q1 - 1.5*(q3-q1) AS lo,
         q3 + 1.5*(q3-q1) AS hi
  FROM dist
)
SELECT
  COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi) AS outliers,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi)/COUNT(*),2) AS pct_outliers
FROM `mgmt-467-2500.netflix.watch_history_dedup` h
CROSS JOIN bounds b;

Query is running:   0%|          |

Downloading:   0%|          |

,outliers,total,pct_outliers
0,3421,99878,3.43


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `mgmt-467-2500.netflix.watch_history_robust` AS
WITH q AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(1)]  AS p01,
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(98)] AS p99
  FROM `mgmt-467-2500.netflix.watch_history_dedup`
)
SELECT
  h.*,
  GREATEST(q.p01, LEAST(q.p99, h.watch_duration_minutes)) AS minutes_watched_capped
FROM `mgmt-467-2500.netflix.watch_history_dedup` h, q;

-- Quantiles before vs after
-- This query is commented out because it's a separate result and should be run in a new cell for clarity.
/*
WITH before AS (
  SELECT 'before' AS which, APPROX_QUANTILES(minutes_watched, 5) AS q
  FROM `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history_dedup`
),
after AS (
  SELECT 'after' AS which, APPROX_QUANTILES(minutes_watched_capped, 5) AS q
  FROM `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history_robust`
)
SELECT * FROM before UNION ALL SELECT * FROM after;
*/

Query is running:   0%|          |

""


### Verification Prompt
Generate a query that shows min/median/max before vs after capping.


In [ ]:
%%bigquery
WITH before AS (
  SELECT
    'before' AS which,
    MIN(watch_duration_minutes) AS min_watched,
    APPROX_QUANTILES(watch_duration_minutes, 2)[OFFSET(1)] AS median_watched,
    MAX(watch_duration_minutes) AS max_watched
  FROM `mgmt-467-2500.netflix.watch_history_dedup`
),
after AS (
  SELECT
    'after' AS which,
    MIN(minutes_watched_capped) AS min_watched,
    APPROX_QUANTILES(minutes_watched_capped, 2)[OFFSET(1)] AS median_watched,
    MAX(minutes_watched_capped) AS max_watched
  FROM `mgmt-467-2500.netflix.watch_history_robust`
)
SELECT * FROM before
UNION ALL
SELECT * FROM after;

Query is running:   0%|          |

Downloading:   0%|          |

,which,min_watched,median_watched,max_watched
0,before,0.2,51.2,799.3
1,after,4.4,51.0,203.0


**Reflection:** When might capping be harmful? Name a model type less sensitive to outliers and why.

### 5.4 Business anomaly flags — What & Why
Human-readable flags help both product decisioning and ML features (e.g., binge behavior).

### Build Prompt
Generate **three BigQuery SQL cells** (adjust if columns differ):
1) In `watch_history_robust`, compute and summarize `flag_binge` for sessions > 8 hours.
2) In `users`, compute and summarize `flag_age_extreme` if age can be parsed from `age_band` (<10 or >100).
3) In `movies`, compute and summarize `flag_duration_anomaly` where `duration_min` < 15 or > 480 (if exists).
Each cell should output count and percentage and include 1–2 comments.


In [ ]:
%%bigquery
-- Summarize flag_binge for sessions > 8 hours in watch_history_robust
-- A session is considered a binge if minutes_watched exceeds 8 hours (480 minutes).
SELECT
  COUNTIF(watch_duration_minutes > 8*60) AS sessions_over_8h,
  COUNT(*) AS total,
  ROUND(SAFE_DIVIDE(100*COUNTIF(watch_duration_minutes > 8*60), COUNT(*)),2) AS pct_flag_binge
FROM `mgmt-467-2500.netflix.watch_history_robust`;

Query is running:   0%|          |

Downloading:   0%|          |

,sessions_over_8h,total,pct_flag_binge
0,638,99878,0.64


In [ ]:
%%bigquery
-- Summarize flag_age_extreme in users
-- Flag users with extreme ages (below 10 or above 100) based on parsing age_band.
SELECT
  COUNTIF(SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR
          SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100) AS extreme_age_rows,
  COUNT(*) AS total,
  ROUND(SAFE_DIVIDE(100*COUNTIF(SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR
                                SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100), COUNT(*)),2) AS pct_flag_age_extreme
FROM `mgmt-467-2500.netflix.users`;

Query is running:   0%|          |

Downloading:   0%|          |

,extreme_age_rows,total,pct_flag_age_extreme
0,895,51500,1.74


In [ ]:
%%bigquery
-- Summarize flag_duration_anomaly in movies
-- Flag movies with unusually short or long durations (less than 15 min or more than 8 hours).
SELECT
  COUNTIF(duration_minutes < 15 OR duration_minutes > 8*60) AS duration_anomaly_titles,
  COUNT(*) AS total,
  ROUND(SAFE_DIVIDE(100*COUNTIF(duration_minutes < 15 OR duration_minutes > 8*60), COUNT(*)),2) AS pct_flag_duration_anomaly
FROM `mgmt-467-2500.netflix.movies`;

Query is running:   0%|          |

Downloading:   0%|          |

,duration_anomaly_titles,total,pct_flag_duration_anomaly
0,115,5200,2.21


### Verification Prompt
Generate a single compact summary query that returns two columns per flag: `flag_name, pct_of_rows`.


In [ ]:
%%bigquery
-- Single compact summary query for anomaly flags
SELECT
  'flag_binge' AS flag_name,
  ROUND(SAFE_DIVIDE(100*COUNTIF(watch_duration_minutes > 8*60), COUNT(*)),2) AS pct_of_rows
FROM `mgmt-467-2500.netflix.watch_history_robust`
UNION ALL
SELECT
  'flag_age_extreme' AS flag_name,
  ROUND(SAFE_DIVIDE(100*COUNTIF(SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR
                                SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100), COUNT(*)),2) AS pct_of_rows
FROM `mgmt-467-2500.netflix.users`
UNION ALL
SELECT
  'flag_duration_anomaly' AS flag_name,
  ROUND(SAFE_DIVIDE(100*COUNTIF(duration_minutes < 15 OR duration_minutes > 8*60), COUNT(*)),2) AS pct_of_rows
FROM `mgmt-467-2500.netflix.movies`;

Query is running:   0%|          |

Downloading:   0%|          |

,flag_name,pct_of_rows
0,flag_binge,0.64
1,flag_age_extreme,1.74
2,flag_duration_anomaly,2.21


**Reflection:** Which anomaly flag is most common? Which would you keep as a feature and why?

## 6) Save & submit — What & Why
Reproducibility: save artifacts and document decisions so others can rerun and audit.

### Build Prompt
Generate a checklist (Markdown) students can paste at the end:
- Save this notebook to the team Drive.
- Export a `.sql` file with your DQ queries and save to repo.
- Push notebook + SQL to the **team GitHub** with a descriptive commit.
- Add a README with your `PROJECT_ID`, `REGION`, bucket, dataset, and today’s row counts.


## Grading rubric (quick)
- Profiling completeness (30)  
- Cleaning policy correctness & reproducibility (40)  
- Reflection/insight (20)  
- Hygiene (naming, verification, idempotence) (10)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [51]:
!git config --global user.name "DanielGallagher1"
!git config --global user.email "gallagherdaniel555@gmail.com"

In [52]:
from getpass import getpass
token = getpass('Enter your GitHub token: ')
!git clone https://{token}@github.com/DanielGallagher1/mgmt467-analytics-portfolio.git

Enter your GitHub token: ··········
fatal: destination path 'mgmt467-analytics-portfolio' already exists and is not an empty directory.


In [53]:
%cd /content/mgmt467-analytics-portfolio/

/content/mgmt467-analytics-portfolio


In [55]:
!cp "/content/drive/My Drive/MGMT467/Labs/Lab4_Regression_BQML.ipynb" "/content/mgmt467-analytics-portfolio/Labs/"

In [57]:
!git add Labs/Lab4_Regression_BQML.ipynb
!git commit -m "Added Lab4_Regression_BQML.ipynb with latest analysis"
!git push https://{token}@github.com/DanielGallagher1/mgmt467-analytics-portfolio.git main

On branch main
Your branch and 'origin/main' have diverged,
and have 4 and 4 different commits each, respectively.
  (use "git pull" to merge the remote branch into yours)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	mgmt467-analytics-portfolio/

nothing added to commit but untracked files present (use "git add" to track)
To https://github.com/DanielGallagher1/mgmt467-analytics-portfolio.git
 ! [rejected]        main -> main (non-fast-forward)
error: failed to push some refs to 'https://github.com/DanielGallagher1/mgmt467-analytics-portfolio.git'
hint: Updates were rejected because the tip of your current branch is behind
hint: its remote counterpart. Integrate the remote changes (e.g.
hint: 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.
